# Tutorial 快速入门

- https://github.com/facebookresearch/faiss/wiki/Faiss-indexes
- https://github.com/facebookresearch/faiss/wiki/Getting-started

For the following, we assume Faiss is installed. We provide code examples in C++ and Python. The code can be run by copy/pasting it or running it from the tutorial/ subdirectory of the Faiss distribution.

## Getting some data

Faiss可以处理固定维度`d`的向量集合，typically a few 10s to 100s. 向量集合被保存在矩阵中。我们假设行主存储（例如，the j'th component of vector number i is stored in row i, column j of the matrix）。Faiss只能使用`32-bit浮点矩阵`。

我们需要两个矩阵：  
- xb 为语料, that contains all the vectors that must be indexed, and that we are going to search in. Its size is nb-by-d
- xq 为查询的向量集合, for which we need to find the nearest neighbors. Its size is nq-by-d. If we have a single query vector, nq=1.

下面例子，我们将学习在`d=64`维空间中由均匀分布所画出的向量。为了增加娱乐性，我们在第一个向量上加个小平移。

In [1]:
import numpy as np

d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

# import matplotlib.pyplot as plt 

# plt.hist(xb[6])
# plt.show()

# Building an index and adding the vectors to it

Faiss is built around the Index object. It encapsulates the set of database vectors, and optionally preprocesses them to make searching efficient. 有很多类型的索引, 我们使用最简单的一个，执行暴力L2距离搜索（brute-force L2 distance search）：`IndexFlatL2`.

所有索引构建时都必须指定向量的维度`d`。而大多数索引还需要一个训练阶段，以便分析向量的分布。对于`IndexFlatL2`来说，可以跳过训练这步（因为是暴力搜索，不用分析向量）.

当构建和训练索引后，在索引上执行两个操作：`add`和`search`.

向索引添加数据，在`xb`上调用`add`方法. 有两个索引的状态变量：
- is_trained, a boolean that indicates whether training is required
- ntotal, 被索引的向量集合的数量

一些索引也可以对每个向量存储整型ID(`IndexFlatL2`不用). 如果不提供ID，使用向量的序号作为id，例如，第一个向量为0，第二个位1，等等。

In [2]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


# Searching

在索引上可以执行的最基本操作是 `k-nearest-neighbor search`, 例如，对每个向量，在数据库中查找它的 `k近邻`.

结果保存在大小为 `nq-by-k` 的矩阵中, 其中，行 `i` 包含了向量`i`的近邻ID, 按距离升序排序. In addition to this matrix, the search operation returns a nq-by-k floating-point matrix with the corresponding squared distances.

先来一个简单检查，用数据库中的小部分向量进行检索，来确保其最近邻确实是向量本身。

In [3]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(xb.shape)
print(I)
print(D)

(100000, 64)
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751733 7.207629  7.2511625]
 [0.        6.3235645 6.684581  6.7999454]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279865 7.6628466]
 [0.        6.7638035 7.2951202 7.3688145]]


In [4]:
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


# Results

In [5]:
The output of the sanity check should look like

SyntaxError: invalid syntax (<ipython-input-5-7a2e4edd0c6d>, line 1)

```text
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
```
```text
[[ 0.          7.17517328  7.2076292   7.25116253]
 [ 0.          6.32356453  6.6845808   6.79994535]
 [ 0.          5.79640865  6.39173603  7.28151226]
 [ 0.          7.27790546  7.52798653  7.66284657]
 [ 0.          6.76380348  7.29512024  7.36881447]]
```
查询结果的确是他自己，相应的距离是0。在下面查询结果的距离矩阵中，按升序排序。

The output of the actual search is similar to

```text
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
```
```text
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]
```
Because of the value added to the first component of the vectors, the dataset is smeared along the first axis in d-dim space. So the neighbors of the first few vectors are around the beginning of the dataset, and the ones of the vectors around ~10000 are also around index 10000 in the dataset.

Executing the search above takes about 3.3s on a 2016 machine.